In [1]:
#for sentence based embeddings -- transformers give token embeddings, we need sentence embeddings
!pip install -qq sentence_transformers
#for working with llms
!pip install -qq -U langchain
#for pdf loading
!pip install -qq -U pypdf
#the vector store
!pip install -qq -U faiss-gpu
#for creating embeddings
!pip install -qq -U InstructorEmbedding
#for the llms
!pip install -qq -U transformers
#for quantized llms
!pip install -qq auto-gptq
!pip install --upgrade --quiet  langchain-core langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
#get drive access
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#load the pdfs
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
#pdfs are located at
pdf_directory = 'path to files'
#pattern of pdfs to load
pdf_pattern = './*.pdf'
#create a loader
loader = DirectoryLoader(
                            pdf_directory,
                            glob = pdf_pattern,
                            loader_cls = PyPDFLoader
)
#load the pdfs
documents = loader.load()

In [5]:
#chunk the pdfs
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20
)
#split pdfs
documents = text_splitter.split_documents(documents)

In [10]:
#create a faiss store
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
                                model_kwargs = {'device' : 'cuda'}
)
vectorstore = FAISS.from_documents(documents = documents, embedding = embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [41]:
#create a model
from transformers import pipeline, AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM
from langchain.llms import HuggingFacePipeline
#load model
model_repo = '4bit/Llama-2-7b-Chat-GPTQ'
model_basename = 'gptq_model-4bit-128g'
model = AutoGPTQForCausalLM.from_quantized(
    model_repo,
    model_basename = model_basename,
    use_safetensors = True,
    trust_remote_code = True,
    #device = 'cuda:0',
    use_triton = False,
    quantize_config = None
)
tokenizer = AutoTokenizer.from_pretrained(model_repo, truncation=True)
pipe = pipeline(
                 task = 'text-generation',
                 model = model,
                 tokenizer = tokenizer,
                 pad_token_id = tokenizer.eos_token_id,
                 max_length = 2048,
                 temperature = 0.5,
                 do_sample = True,
                 top_p = 0.95,
                 repetition_penalty = 1.15
)
llm_model = HuggingFacePipeline(pipeline = pipe)

INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralFo

In [85]:
#writing custom since truncation is not working
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
number_of_questions = 5
#generate number_of_questions questions and store in a list
template = '''
              You are given a question below. Generate a similar question from it about the
              same question. Ask quite creatively, include what if situations and speculations too. But ask only question.
              input: {question}----------
'''
prompt = PromptTemplate.from_template(template)
rewriter = prompt | llm_model | StrOutputParser()
original_question = 'What is the true strength of a wizard?'
questions = [original_question]
for qdx in range(number_of_questions - 1):
    rephrased_question = rewriter.invoke(original_question).split('----------')[1]
    questions.append(rephrased_question)

In [86]:
for i, question in enumerate(questions):
  print(i, question)

0 What is the true strength of a wizard?
1 

              Question: What if the true strength of a wizard lies not in their magical prowess but in their ability to connect with nature and harness its power? Imagine a world where magic and technology coexist, how would you design a spell that could tap into the earth's core energy to create an unlimited source of clean energy?
2 
Question: What if the true strength of a wizard lies not in their ability to cast spells but in their capacity to brew the most potent elixirs? Imagine a world where magic users compete in a high-stakes cocktail party, with each concoction more delicious than the last. How would you fare in such a competition? Would your enchanted smoothies be able to outshine even the mightiest of wizards?
3 
question: If a wizard were to cast a spell that could grant him immense power but also had the side effect of slowly turning him into stone, how long would he have to keep casting the spell before he turned completely? W

In [87]:
#get context for all the questions that we got
chunks = ""
sources = set([])
for question in questions:
  retrieved = vectorstore.similarity_search(question, k=10)
  for chunk in retrieved:
    chunks = chunks + chunk.page_content + " "
    sources.add(chunk.metadata['source'].split(' - ')[2])

In [92]:
question_template = '''You are to answer a question based on the context given below. Answer based only
on the context. If you cannot answer based only the context, reply that you cannot answer this question.
context:{context}
question:{question}
answer:$$answer$$
'''
question_prompt = PromptTemplate.from_template(question_template)
answer = question_prompt | llm_model | StrOutputParser()
s = answer.invoke({'context' : chunks, 'question' : original_question}).split('$$answer$$')[1]
print(s)


based solely on the context provided, here is what I can say:
* The passage mentions several powerful wizards, including the "greatest wizard of all time" and the "most powerful wizard in the world."
* The speaker claims to have been more powerful than any wizard alive during a particular period in the past.
* The speaker refers to their own power as "immensity," implying that it is significant.
* The passage suggests that defeating the greatest wizard of all time is a notable achievement, indicating that the wizard's strength is substantial.
* The speaker compares themselves favorably to another wizard, stating that they were stronger than the wizard in question.
* The passage implies that the ability to create powerful spells and potions is a sign of a strong wizard.
* The speaker seems proud of their accomplishments as a wizard, suggesting that they consider their strength to be impressive.

In conclusion, based on the context alone, we can infer that the true strength of a wizard 